In [1]:
import io
import os
import sys
import types
import numpy as np
import pandas as pd
import altair as alt
import dropbox

In [2]:
%run MakeNBImportAvailable.ipynb

In [3]:
from datetime import date
from datetime import datetime, timedelta
# import RetrieveData as rd_gn
# import knmi_gn_001 as gethdd
# import Process_Data_CH as prcs
import GetMySQL

importing Jupyter notebook from GetMySQL.ipynb


In [4]:
settings = GetMySQL.readsettings()

In [5]:
poi = GetMySQL.SetPOI('vandaag')

POI is :  {'start': '2022-06-13', 'end': '2022-06-14'}


In [6]:
df_EH = GetMySQL.getSQL(poi,'item0052',settings,'EH')
df_EH = GetMySQL.RemoveReadingErrors(df_EH)
df_EL = GetMySQL.getSQL(poi,'item0051',settings,'EL')
df_EL = GetMySQL.RemoveReadingErrors(df_EL)
df_g = GetMySQL.getSQL(poi,'item0054',settings,'gas')
df_g = GetMySQL.RemoveReadingErrors(df_g)

df_E =pd.concat([df_EH , df_EL])

bereken verbruik voor EH en EL  
combineer EH en EL verbruik tot 1 data, hanteer label EH EL per tijdstap  

Dan gasverbruik er onder plakken  
cumulatief verbruik voor zowel E als G  
moving gemiddeld in W voor E en in ??   voor G  


verbruik is verschil in meterstand in kWh
tussenpoos is een minuut
gemiddeld vermogen is verbruik in Wh / (1/60 hr)

In [7]:
# Electriciteit tot 1 df met label EH, EL per tijdstap
df_E['verbruik'] = df_E[['counter','variable']].groupby(['variable']).diff().shift(-1)
df_E = df_E.fillna(0)

In [8]:
table = pd.pivot_table(df_E, values='verbruik', index=['datetime'],
                    columns=['variable'], aggfunc=np.sum)
table['variable']='EL'
table.loc[table['EH']!=0,'variable']='EH'
table

variable                EH     EL variable
datetime                                  
2022-06-13 00:00:00  0.000  0.000       EL
2022-06-13 00:01:00  0.000  0.000       EL
2022-06-13 00:02:00  0.000  0.012       EL
2022-06-13 00:03:00  0.000  0.004       EL
2022-06-13 00:04:00  0.000  0.000       EL
...                    ...    ...      ...
2022-06-13 20:23:00  0.005  0.000       EH
2022-06-13 20:24:00  0.006  0.000       EH
2022-06-13 20:25:00  0.007  0.000       EH
2022-06-13 20:26:00  0.007  0.000       EH
2022-06-13 20:27:00  0.000  0.000       EL

[1228 rows x 3 columns]

In [9]:
table['verbruik'] = table['EH']+table['EL']
table.head()

variable              EH     EL variable  verbruik
datetime                                          
2022-06-13 00:00:00  0.0  0.000       EL     0.000
2022-06-13 00:01:00  0.0  0.000       EL     0.000
2022-06-13 00:02:00  0.0  0.012       EL     0.012
2022-06-13 00:03:00  0.0  0.004       EL     0.004
2022-06-13 00:04:00  0.0  0.000       EL     0.000

In [10]:
df_g['verbruik'] =df_g[['counter','variable']].groupby(['variable']).diff().shift(-1)
df_g = df_g.fillna(0)

In [11]:
cols = ['variable','verbruik']
df = pd.concat([df_g[cols],table[cols]])

In [12]:
df['verbruik_W'] = df['verbruik']*1000 / (1/60)
df.loc[df['variable']=='gas','verbruik_W']=0

In [13]:
# df['verbruik_W'] = df['verbruik']*1000 * 60
# df['moving'] = df.groupby('variable')['verbruik_W'].transform(lambda x: x.rolling(10, 1).mean())

df['cumul'] = df[['variable','verbruik']].groupby(['variable']).cumsum()

In [14]:
source = df.reset_index()
source.dtypes

datetime      datetime64[ns]
variable              object
verbruik             float64
verbruik_W           float64
cumul                float64
dtype: object

In [15]:
domain = ['EH', 'EL', 'gas']
range_ = ['red', 'green', 'blue']



In [16]:
wdth = 300
chart1a = alt.Chart(source[source['variable']!='gas'],title='elektriciteit').mark_bar(
        strokeWidth=1,point=True).encode(
        x=alt.X('datetime:T'),
        y=alt.Y("verbruik_W:Q",
            axis=alt.Axis(title="gemiddeld vermogen [W]")),
    color=alt.Color('variable:N',scale=alt.Scale(domain=domain,range=range_))
    ).properties(width=wdth,height=250).interactive()
chart1b = alt.Chart(source[source['variable']=='gas'],title='gas').mark_bar(
        strokeWidth=1,point=True).encode(
        x=alt.X('datetime:T'),
        y=alt.Y("verbruik:Q",
            axis=alt.Axis(title="verbruik [m3]")),
    color=alt.Color('variable:N')
    ).properties(width=wdth,height=250).interactive()
# chartm = alt.Chart(source).mark_line(strokeWidth=2).encode(
#      x=alt.X('datetime:T'),
#     y=alt.Y("moving:Q"),
#     color=alt.Color('variable:N')
#     ).properties(width=400,height=250).interactive()
# chart1 = chart1 + chartm

chart2a = alt.Chart(source[source['variable']!='gas'],
                           title='elektriciteit cumulatief').mark_line(
        strokeWidth=1).encode(
        x=alt.X('datetime:T',
            axis=alt.Axis(title="tijd")),
        y=alt.Y("cumul:Q",
            axis=alt.Axis(title="verbruik [kWh]")),
    color=alt.Color('variable:N')
    ).properties(width=wdth,height=250).interactive()


chart2b = alt.Chart(source[source['variable']=='gas'],
                    title='gas cumulatief').mark_line(
        strokeWidth=1).encode(
        x=alt.X('datetime:T'),
        y=alt.Y("cumul:Q",
            axis=alt.Axis(title="verbruik [m3]")),
    color=alt.Color('variable:N')
    ).properties(width=wdth,height=250).interactive()

chart_all=alt.vconcat(chart1a|chart1b,chart2a|chart2b)
chart_all



alt.VConcatChart(...)

In [17]:
df.groupby(['variable']).resample('D').sum().reset_index()

variable   datetime  verbruik  verbruik_W     cumul
0       EH 2022-06-13     5.676    340560.0  1646.430
1       EL 2022-06-13     1.619     97140.0   573.650
2      gas 2022-06-13     0.403         0.0   162.027